# Preprocess

## Imports

In [1]:
import pandas as pd
from IPython.display import clear_output

In [2]:
%load_ext autoreload
%autoreload 2
clear_output()

In [3]:
from Dude import table

## Varriables

In [7]:
trips

,route_id,trip_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,boarding_door
0,78,B864691,B86469AHCMAA-081,"Újpalota, Nyírpalota út",0,B86469_2770_1_0,V014,1.0,2.0,2.0
1,78,B864711,B86471APPMAA-081,"Újpalota, Nyírpalota út",0,B86471_2770_1_0,V014,1.0,2.0,NaN
2,2240,B945231,B94523ASZSzN-041,"Újpalota, Szentmihályi út",0,B94523_2240_50_0,MW15,1.0,2.0,NaN
3,2240,B9452310,B94523ASZSzN-041,"Újpalota, Szentmihályi út",0,B94523_2240_50_8,MW15,1.0,2.0,NaN
4,2240,B9452311,B94523ASZSzN-041,"Újpalota, Szentmihályi út",0,B94523_2240_50_9,MW15,1.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...
221746,H5,H8719_25,8281,Batthyány tér,1,NaN,NaN,NaN,NaN,NaN
221747,H5,H8720_29,8281,Békásmegyer,0,NaN,NaN,NaN,NaN,NaN
221748,H5,H8729_25,8281,Batthyány tér,1,NaN,NaN,NaN,NaN,NaN
221749,H5,H8730_29,8289,Békásmegyer,0,NaN,NaN,NaN,NaN,NaN


In [10]:
#stop_time_trips = pd.merge(trips,stop_times, on="trip_id")
df = mgm(trips,stop_times)

In [11]:
df.head()

,trip_id,route_id,stop_sequence,stop_id,direction_id
0,B864691,78,0,008569,0
1,B864691,78,1,F03291,0
2,B864691,78,2,F03403,0
3,B864691,78,3,F03402,0
4,B864691,78,4,F03400,0


In [31]:
print(
    f"""
    Number of identical routes: {stop_time_trips.route_id.unique().size}
    """
)


    Number of identical routes: 519
    


In [42]:
nodes = df.stop_id.value_counts()